In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18, 12)

In [5]:
products = pd.read_csv('NL Estimation Set.csv')
products = products[products.uniquequarter == 2]

# Converting boolean variable to integer
products['nonstop'] = products.nonstop.astype('int')

# Multiplying number of passengers by 10 to get the quantity that the DB1B passengers are representing
products['passengers'] = products.passengers * 10
# Differentiating between nonstop and direct flights
products['Passengers (nonstop)'] = np.where(products.nonstop == 1, products.passengers, np.nan)

# Variables for product level descriptives, with cleaned up names
sumvars = pd.DataFrame(
    np.array(products[['price', 'passengers', 'Passengers (nonstop)', 'nonstop',
                       'market_miles_flown', 'extra_miles', 'presence_origin', 'presence_dest']]),
    columns = ['Price', 'Passengers', 'Passengers (nonstop)', 'Nonstop (dummy)', 'Total distance',
               'Extra Miles', 'Origin presence', 'Destination presence']
).describe().T.iloc[:,[1,2,3,7,5]].round(2)
sumvars.columns = ['Mean', 'Std. Dev.', 'Min', 'Max', 'Median']

# Aggreting to market level for market level descriptives
products['Number of products'] = products.groupby('market_id')['market_id'].transform('count')
products['Number of airlines'] = products.groupby('market_id')['ticket_carrier'].transform('nunique')
products['Market passengers'] = products.groupby('market_id')['passengers'].transform('sum')
markets = products[['Number of products', 'Number of airlines', 'Market passengers', 'market_size',
                    'market_id']].drop_duplicates().rename(columns = {'market_size' : 'Market size'}).drop('market_id', axis = 1)
markets = markets.describe().T.iloc[:,[1,2,3,7,5]].round(2)
markets.columns = ['Mean', 'Std. Dev.', 'Min', 'Max', 'Median']

In [6]:
pd.concat([sumvars, markets], axis = 0)

,Mean,Std. Dev.,Min,Max,Median
Price,274.82,123.79,14.00,1250.00,249.72
Passengers,453.91,2917.72,10.00,114780.00,40.00
Passengers (nonstop),4160.04,9422.78,10.00,114780.00,360.00
Nonstop (dummy),0.08,0.27,0.00,1.00,0.00
Total distance,1489.63,808.32,152.00,7445.00,1305.00
Extra Miles,1.24,0.31,1.00,2.99,1.11
Origin presence,0.84,0.17,0.01,1.00,0.90
Destination presence,0.84,0.16,0.00,1.00,0.90
Number of products,22.44,15.73,1.00,162.00,18.00
Number of airlines,4.32,1.57,1.00,12.00,4.00


In [7]:
pd.concat([sumvars, markets], axis = 0).to_excel('Table 1 - summary stats.xlsx')